# 导入相应的库

In [18]:
import requests
from fake_useragent import UserAgent
from lxml import etree
import random
import pymysql
import time

# 创建MySQL的表格

In [19]:
connection = ''
connection = pymysql.connect(host='localhost',port=3306,user='root',password='chen3322111',db='doubantop',charset='utf8')
curosr = connection.cursor() #创建一个游标对象
sql ='''
    create table doubantop250(
        id int PRIMARY KEY auto_increment, 
        name TEXT,
        director TEXT,
        year TEXT,
        score TEXT,
        num TEXT,
        introduction TEXT)
    '''
curosr.execute(sql) #执行创建表格            

0

# 进行爬取的反爬的处理

In [20]:
ua = UserAgent()
with open(r'C:\Users\SilentCat\ip.txt','r')as f:#该文件是在快代理爬取的一些代理地址(只爬取到一页就被封了)
    data = f.readlines()
    f.close()
reg = []
for i in data:
    k = eval(i)
    reg.append(k)
ip = random.choice(reg) #在文件随机选取一个代理地址
proxies = {
    "http":'ip'
}
headers = {
    'User-Agent':ua.random,
    'Referer':'https://movie.douban.com/top250?start=0&filter='
}

# 定义一个函数爬取豆瓣top250页面中相应电影的详细网址

In [21]:
def get_detail_url(url):
    response = requests.get(url=url,headers=headers,proxies=proxies)
    if response.status_code == 200:
        page_text = response.text
        html = etree.HTML(page_text)
        detail_url = html.xpath('.//div[@class="hd"]/a/@href') #获取队形电影的详情页面
        print(detail_url)
        return detail_url
    else:
        return False 

# 定义一个爬取相应电影数据的函数#

In [25]:
def parse_detail_url(detail_url):
    response = requests.get(url=detail_url,headers=headers,proxies=proxies)
    if response.status_code == 200:
        page_text = response.text
        html = etree.HTML(page_text)
        #电影标题
        Movie_title = html.xpath('.//h1/span[@property="v:itemreviewed"]/text()')[0]
        print(Movie_title) #用于查看是否进行正常的爬取
        #导演
        Director = html.xpath('//div[@id="info"]/span[1]/span[2]/a/text()')[0]
        #上映年份
        Year = html.xpath('.//*[@id="content"]/h1/span[2]/text()')
        #豆瓣评星
        Score = html.xpath('.//div[@id="interest_sectl"]//strong[@class="ll rating_num"]/text()')
        #影评人数
        num_review = html.xpath('.//div[@id="interest_sectl"]//div[@class="rating_sum"]/a/span/text()')
        #剧情简介
        Introduction = ''.join(html.xpath("//div[@class='related-info']/div[@id='link-report']/span/text()")).strip().replace(' ', '').replace('\n', '').replace('\xa0', '').replace(u'\u3000', u' ')
        if len(Introduction) == 0:
            Introduction = ''.join(html.xpath('.//*[@id="link-report"]/span[1]/text()[1]')).strip().replace(' ', '').replace('\n', '').replace('\xa0', '').replace(u'\u3000', u' ')
        
        #存储在MySQL创建的表格里
        curosr.execute ("insert into doubantop250(name,director,year,score,num,introduction)" "values ('%s','%s','%s','%s','%s')",(str(Movie_title),str(Director),str(Year),str(Score),str(num_review),str(Introduction)))
        connection.commit() #提交修改
        connection.rollback()
        connection.close()
        time.sleep(2) #这里每爬取到一个电影的数据就暂停一下，防止爬取速度过快
        
    else:
        return False

# 定义一个主函数调取前两个函数

In [26]:
def snatch():
    url = 'https://movie.douban.com/top250?start={}&filter='
    for i in range(10):
        top250_url = url.format(i*25)
        detail_urls = get_detail_url(top250_url)
        if detail_urls == 0:
            print("出错啦\n")
            return False
        for detail_url in detail_urls:
            parse_detail_url(detail_url)
        print("一页数据已经爬取完了\n")
        time.sleep(40) #防止爬取过快被封ip地址
        

In [27]:
if __name__ == "__main__":
    snatch()

['https://movie.douban.com/subject/1292052/', 'https://movie.douban.com/subject/1291546/', 'https://movie.douban.com/subject/1292720/', 'https://movie.douban.com/subject/1295644/', 'https://movie.douban.com/subject/1292722/', 'https://movie.douban.com/subject/1292063/', 'https://movie.douban.com/subject/1291561/', 'https://movie.douban.com/subject/1295124/', 'https://movie.douban.com/subject/3541415/', 'https://movie.douban.com/subject/3011091/', 'https://movie.douban.com/subject/1889243/', 'https://movie.douban.com/subject/1292064/', 'https://movie.douban.com/subject/1292001/', 'https://movie.douban.com/subject/3793023/', 'https://movie.douban.com/subject/2131459/', 'https://movie.douban.com/subject/1291549/', 'https://movie.douban.com/subject/1292213/', 'https://movie.douban.com/subject/25662329/', 'https://movie.douban.com/subject/1307914/', 'https://movie.douban.com/subject/5912992/', 'https://movie.douban.com/subject/1291841/', 'https://movie.douban.com/subject/1849031/', 'https:/

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '肖申克的救赎 The Shawshank Redemption'',''弗兰克·德拉邦特'',''[\\'(' at line 1")